In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm

In [ ]:
# Load our data.
df = pd.read_csv("data/Normalized_Purchase_Card_Transactions.csv")
df_agencies = pd.read_csv("data/List_Agencies.csv")
df_vendors = pd.read_csv("data/List_Vendors.csv")

In [ ]:
blank = np.zeros((df['AGENCY_NUM'].max() + 1, df['VENDOR_NAME_NUM'].max() + 1))

for index, row in tqdm(df.iterrows(), total = df.shape[0]):
    blank[row['AGENCY_NUM'], row['VENDOR_NAME_NUM']] += row['TRANSACTION_AMOUNT_NORM']

np.count_nonzero(blank)

In [ ]:
wordCounts = pd.DataFrame(blank)

bags = {i: df[df['AGENCY_NUM'] == i]['VENDOR_NAME_NUM'].unique() for i in df['AGENCY_NUM'].unique()}

In [ ]:
def tf(counts):
    tfDF = pd.DataFrame()
    
    for index, row in counts.iterrows():
        tfDF[index] = row.div(row.sum())
        
    return tfDF

tf = pd.DataFrame(tf(wordCounts)).T

In [ ]:
idf = tf.apply(lambda x: np.log((df['AGENCY_NUM'].max() + 1) / (np.count_nonzero(x.to_numpy())+1)), axis=0)

In [ ]:
prod = tf*idf

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(prod).astype("float")
for i in range(len(similarity)):
    similarity[i][i] = 0

In [ ]:
# Find most similar vendors
mostsimilar = np.argmax(similarity, axis=1)
mostsimilar_verbose = np.array([df_agencies.iloc[i][1] for i in mostsimilar])
mostsimilar_vendors = np.array([bags[i] for i in mostsimilar], dtype = object)

In [ ]:
grouped = df.groupby(['AGENCY_NUM', 'VENDOR_NAME_NUM'], as_index = False)['TRANSACTION_AMOUNT'].agg('sum')
topn = grouped.sort_values(['AGENCY_NUM','TRANSACTION_AMOUNT'], ascending=False).groupby('AGENCY_NUM').head(5)

In [ ]:
descriptions = df.loc[:, ['VENDOR_NAME_NUM', 'TRANSACTION_AMOUNT', 'MCC_DESCRIPTION']]
descriptions['MCC_DESCRIPTION'] = descriptions['MCC_DESCRIPTION'].str.replace('Not Elsewhere Classified', '').str.replace('NOT ELSEWHERE CLASSIFIED', '')
descriptions['MCC_DESCRIPTION_LIST'] = descriptions['MCC_DESCRIPTION'].str.split(r',\s*(?![^()]*\))')

In [ ]:
exploded = descriptions.explode('MCC_DESCRIPTION_LIST').replace(' ', np.nan).dropna(0)

In [ ]:
grouped_descriptions = exploded.groupby('MCC_DESCRIPTION_LIST').sum('TRANSACTION_AMOUNT')
topn_descriptions = pd.DataFrame(grouped_descriptions['TRANSACTION_AMOUNT'].groupby('MCC_DESCRIPTION_LIST', group_keys = False).nlargest(25))

In [ ]:
# TOP N
topn

In [ ]:
# topn_descriptions
descriptions

In [ ]:
descriptions_per_vendor = {}
for i in range(df_agencies.shape[0]):
    top_5_vendors = topn[topn['AGENCY_NUM'] == i]['VENDOR_NAME_NUM'].to_numpy()
    
    desc = []
    for j in top_5_vendors:
        desc += np.unique(descriptions[descriptions['VENDOR_NAME_NUM'] == j]['MCC_DESCRIPTION_LIST'].to_numpy()).tolist()[0]
        
    descriptions_per_vendor[i] = desc

#print(descriptions_per_vendor)

In [ ]:
def summarize(agency = 0):
    """
    Gives brief information on the agency and its most similar agency.
    """
    agency_bag = bags[agency]
    agency_bag_verbose = np.array([df_vendors.iloc[i][1] for i in agency_bag])
    
    similar_agency_bag = bags[mostsimilar[agency]]
    similar_agency_bag_verbose = np.array([df_vendors.iloc[i][1] for i in similar_agency_bag])
    
    print(similar_agency_bag)
    
    for i in similar_agency_bag:
        df[df['MCC_DESCIPTION'].str.contains()]
    
    
    
    print("Selected Agency:", df_agencies.iloc[agency][1])
    print("Most Similar Agency:", mostsimilar_verbose[agency])
    print()
    print("Agency Vendors:", agency_bag_verbose)
    print()
    print("Most Similar Agency Vendors:", similar_agency_bag_verbose)

In [ ]:
summarize(0)


In [ ]:
#df[df['MCC_DESCRIPTION'].str.contains('\(Sports')]
df[(df['AGENCY_NUM'] == 0) & (df['VENDOR_NAME_NUM'] == 707)]